In [1]:
from bs4 import BeautifulSoup as bs
import requests as req
import os
import time
import threading
from colorama import Fore, Back, Style
import weasyprint
from lxml import etree

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'}


In [4]:
fails = []
# tries = ['https://teachmeanatomy.info/the-basics/', 'https://teachmeanatomy.info/head/', 'https://teachmeanatomy.info/neuroanatomy/', 'https://teachmeanatomy.info/neck/', 'https://teachmeanatomy.info/thorax/',
#  'https://teachmeanatomy.info/back/', 'https://teachmeanatomy.info/upper-limb/', 'https://teachmeanatomy.info/lower-limb/', 'https://teachmeanatomy.info/abdomen/', 'https://teachmeanatomy.info/pelvis/', ]
tries = ["https://teachmephysiology.com/subjects/"]
home_ = "https://teachmephysiology.com/"
success = []
threadPool = []
storage = "physio/"


In [5]:
def saveFile(filePath, data):
    try:
        print(Fore.GREEN, "Starting saving {}".format(
            filePath), Style.RESET_ALL)
        weasyprint.HTML(string=data).write_pdf(
            storage+filePath+'.pdf', presentational_hints=True)
        time.sleep(1)
        print(Fore.GREEN, "Finished saving {}".format(
            filePath), Style.RESET_ALL)
        success.append(home_+filePath)
    except Exception as e:
        print(e)
        print(Fore.RED, "Failed to save {}".format(filePath), Style.RESET_ALL)
        fails.append(home_+filePath)


In [6]:
while True:
    url = tries[0]

    print(url)
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    print(Back.RED, 'start time : ', current_time, Style.RESET_ALL)

    url = url.strip('/')
    URL = url

    webpage = req.get(URL, headers=headers)
    soup = bs(webpage.content, "html.parser")
    dom = etree.HTML(str(soup))

    listTarget = dom.xpath('//*[@id="mainContent"]/div[3]')
    contentTarget = dom.xpath('//*[@id="mainContent"]/div[3]/article')

    new_o = []

    if len(contentTarget):
        filePath = url.split('/', 3)[-1]
        parent = filePath.rsplit('/', 1)[0]

        if not os.path.exists(storage+parent):
            os.makedirs(storage+parent)

        threadN = threading.Thread(
            target=saveFile, args=(filePath, webpage.text))
        threadN.start()
        threadPool.append(threadN)
        time.sleep(10)

    else:
        newDomString = etree.tostring(listTarget[0])
        newSoup = bs(newDomString, "html.parser")
        links = newSoup.findAll('a')

        if len(links):
            for link in links:
                linkX = link.get('href')
                if 'partnerprogramm' not in linkX and 'teachmephysiology' in linkX:
                    new_o.append(link.get('href'))
            success.append(url)
            print(Fore.GREEN, "Finished getting lists from {}".format(
                url), Style.RESET_ALL)
        else:
            listTarget = dom.xpath('//*[@id="mainContent"]/div[4]')

            if len(listTarget):
                newDomString = etree.tostring(listTarget[0])
                newSoup = bs(newDomString, "html.parser")
                links = newSoup.findAll('a')

                for link in links:
                    linkX = link.get('href')
                    if 'partnerprogramm' not in linkX:
                        new_o.append(link.get('href'))
                success.append(url)
                print(Fore.GREEN, "Finished getting lists from {}".format(
                    url), Style.RESET_ALL)
            else:
                print(Fore.RED, "Failed to get lists in {}".format(
                    url), Style.RESET_ALL)
                fails.append(url)

        tries.extend(new_o)
    # break

    tries.pop(0)
    if not len(tries):
        break

    print("Remaining files {}".format(len(tries)))

for thread in threadPool:
    thread.join()


https://teachmephysiology.com/subjects/
 start time :  13:25:51 
 Finished getting lists from https://teachmephysiology.com/subjects 
Remaining files 10
https://teachmephysiology.com/histology/
 start time :  13:25:53 
 Finished getting lists from https://teachmephysiology.com/histology 
Remaining files 11
https://teachmephysiology.com/biochemistry/
 start time :  13:25:54 
 Finished getting lists from https://teachmephysiology.com/biochemistry 
Remaining files 15
https://teachmephysiology.com/cardiovascular-system/
 start time :  13:25:56 
 Finished getting lists from https://teachmephysiology.com/cardiovascular-system 
Remaining files 18
https://teachmephysiology.com/respiratory-system/
 start time :  13:25:57 
 Finished getting lists from https://teachmephysiology.com/respiratory-system 
Remaining files 20
https://teachmephysiology.com/gastrointestinal-system/
 start time :  13:25:59 
 Finished getting lists from https://teachmephysiology.com/gastrointestinal-system 
Remaining files

In [8]:
import random
import shutil
files_ = os.popen("find ./{} -name '*.pdf'".format(storage)
                  ).read().strip().split('\n')


for file_ in files_:
    name_ = file_.rsplit('/', 1)[-1].replace('.pdf', "").capitalize() + \
        str(random.randint(1000, 9999))+'.pdf'
    shutil.copy2(file_, './all{}/'.format(storage.capitalize())+name_)
